In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=8447779df7ca664d5ac9e5c41b03446107fb9ebe5dc85e58cdb49da3952047d3
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Load data from CSV files into Spark DataFrames
user_df = spark.read.csv("/content/sample_data/user_dim.csv", header=True, inferSchema=True)
transaction_df = spark.read.csv("/content/sample_data/transaction_fact.csv", header=True, inferSchema=True)
fraud_indicators_df = spark.read.csv("/content/sample_data/fraud_indicators.csv", header=True, inferSchema=True)

# Show data to verify
user_df.show()
transaction_df.show()
fraud_indicators_df.show()


+-------+---------+-------------------+-----------+---------------------+
|user_id|user_name|         user_email|   location|account_creation_date|
+-------+---------+-------------------+-----------+---------------------+
|      1|    Alice|  alice@example.com|   New York|           2020-01-15|
|      2|      Bob|    bob@example.com|Los Angeles|           2020-03-22|
|      3|  Charlie|charlie@example.com|    Chicago|           2021-05-10|
|      4|    David|  david@example.com|    Houston|           2019-07-30|
|      5|      Eva|    eva@example.com|    Phoenix|           2022-08-14|
+-------+---------+-------------------+-----------+---------------------+

+--------------+-------+------------------+-------------------+-------------------+--------------+-----------+-----------+
|transaction_id|user_id|transaction_amount|   transaction_date|   transaction_time|payment_method|   location|fraud_label|
+--------------+-------+------------------+-------------------+-------------------+----

In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Create Spark session
spark = SparkSession.builder.appName("FraudDetection").getOrCreate()

# Load static data from CSV files
transaction_df = spark.read.csv("/content/sample_data/transaction_fact.csv", header=True, inferSchema=True)
user_df = spark.read.csv("/content/sample_data/user_dim.csv", header=True, inferSchema=True)
fraud_indicators_df = spark.read.csv("/content/sample_data/fraud_indicators.csv", header=True, inferSchema=True)

# Detect anomalies from static data
anomalies = transaction_df.filter(
    (col("transaction_amount") > 1000) |
    (col("location") != col("location"))
)

# join with user_df and transaction_df to enrich data
enriched_anomalies = anomalies.join(user_df, "user_id", "left") \
                               .join(transaction_df, "transaction_id", "left")

# Show results
enriched_anomalies.show()





+--------------+-------+------------------+-------------------+-------------------+--------------+--------+-----------+---------+-----------------+--------+---------------------+-------+------------------+-------------------+-------------------+--------------+--------+-----------+
|transaction_id|user_id|transaction_amount|   transaction_date|   transaction_time|payment_method|location|fraud_label|user_name|       user_email|location|account_creation_date|user_id|transaction_amount|   transaction_date|   transaction_time|payment_method|location|fraud_label|
+--------------+-------+------------------+-------------------+-------------------+--------------+--------+-----------+---------+-----------------+--------+---------------------+-------+------------------+-------------------+-------------------+--------------+--------+-----------+
|           101|      1|            1200.0|2023-09-01 00:00:00|2024-09-12 10:00:00|   Credit Card|New York|          0|    Alice|alice@example.com|New Yor